#  Invoice Data Extraction Process using Document AI

Document AI is a Google Cloud platform tool that takes unstructured data from documents and transforms it into structured data, making it easier to understand, analyze, and consume. Document AI uses natural language processing, computer vision and Google Cloud services to help you create scalable, end-to-end, cloud-based document processing applications.

To know more about Document AI. Follow this 

https://medium.com/@monishmoz1/streamline-data-extraction-with-googles-document-ai-4c92fe280344

In [20]:
!pip install pyarrow 
!pip install google-cloud-documentai

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [12]:
!pip install google-cloud-documentai

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for google-cloud-documentai from https://files.pythonhosted.org/packages/56/f1/8c245a3aa07eb7fc478a559dd643055a2ce7aba1b9e930df1d1a99ac4ac0/google_cloud_documentai-2.23.0-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/305.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/305.9 kB ? eta -:--:--
   --- ----------------------------------- 30.7/305.9 kB 435.7 kB/s eta 0:00:01
   ------- ------------------------------- 61.4/305.9 kB 469.7 kB/s eta 0:00:01
   ---------- ---------------------------- 81.9/305.9 kB 512.0 kB/s eta 0:00:01
   ------------- ------------------------ 112.6/305.9 kB 504.4 kB/s eta 0:00:01
   -------------------- ----------------- 163.8/305.9 kB 614.4 kB/s eta 0:00:01
   ------------------------ ------------- 194.6/305.9 kB 655.9 kB/s eta 0:00:01
   ------------------------------ ------- 245.8/305.9 kB 686.6 

In [42]:
# import pyarrow.parquet as pq

# # Specify the file path of the Parquet file
# file_path = "./train-00000-of-00001-76ffc8319f74dd65.parquet"

# # Read the Parquet file
# table = pq.read_table(file_path)

# # Convert the table to a Pandas DataFrame (optional)
# df = table.to_pandas()

# # Now you can work with the DataFrame as you would normally in Pandas
# print(df.head())


In [ ]:
from huggingface_hub import login
login()

In [32]:
from datasets import load_dataset

dataset = load_dataset("mychen76/invoices-and-receipts_ocr_v1")

Generating train split:   0%|          | 0/2043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/125 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/70 [00:00<?, ? examples/s]

In [41]:
df = dataset['train'].to_pandas()

In [37]:
df.columns

Index(['image', 'id', 'parsed_data', 'raw_data'], dtype='object')

In [4]:
df.shape

(2043, 4)

In [5]:
import json

json.loads(df['parsed_data'][0])['json']

'{\'header\': {\'invoice_no\': \'40378170\', \'invoice_date\': \'10/15/2012\', \'seller\': \'Patel, Thompson and Montgomery 356 Kyle Vista New James, MA 46228\', \'client\': \'Jackson, Odonnell and Jackson 267 John Track Suite 841 Jenniferville, PA 98601\', \'seller_tax_id\': \'958-74-3511\', \'client_tax_id\': \'998-87-7723\', \'iban\': \'GB77WRBQ31965128414006\'}, \'items\': [{\'item_desc\': "Leed\'s Wine Companion Bottle Corkscrew Opener Gift Box Set with Foil Cutter", \'item_qty\': \'1,00\', \'item_net_price\': \'7,50\', \'item_net_worth\': \'7,50\', \'item_vat\': \'10%\', \'item_gross_worth\': \'8,25\'}], \'summary\': {\'total_net_worth\': \'$7,50\', \'total_vat\': \'$0,75\', \'total_gross_worth\': \'$8,25\'}}'

In [11]:
from PIL import Image
import io

# Assuming image_bytes contains the bytes of the image

# Create a BytesIO object to wrap the image bytes
image_stream = io.BytesIO(df['image'][0]['bytes'])

# Open the image using PIL's Image module
image = Image.open(image_stream)

# Display the image
image.show()


In [6]:
from typing import Optional

from google.api_core.client_options import ClientOptions
from google.cloud import documentai  # type: ignore



def process_document_sample(
    project_id: str,
    location: str,
    processor_id: str,
    image_content,
    mime_type: str,
    field_mask: Optional[str] = None,
    processor_version_id: Optional[str] = None,
) -> None:
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    if processor_version_id:

        name = client.processor_version_path(
            project_id, location, processor_id, processor_version_id
        )
    else:
        # The full resource name of the processor, e.g.:
        # `projects/{project_id}/locations/{location}/processors/{processor_id}`
        name = client.processor_path(project_id, location, processor_id)

    # Read the file into memory
#     with open(file_path, "rb") as image:
#         image_content = image.read()

    raw_document = documentai.RawDocument(content=image_content, mime_type=mime_type)


    process_options = documentai.ProcessOptions(
        individual_page_selector=documentai.ProcessOptions.IndividualPageSelector(
            pages=[1]
        )
    )

    request = documentai.ProcessRequest(
        name=name,
        raw_document=raw_document,
        field_mask=field_mask,
        process_options=process_options,
    )

    result = client.process_document(request=request)


    document = result.document

    return document

## General - Form Parser Processor

In [8]:
import time

project_id = ""
location = "us"
processor_id = ""      #provide processor id here
mime_type = "image/png"
field_mask = "text,entities,pages.pageNumber"
processor_version_id = ""

document_result_obj = []



start = time.time()

## To iterate through the dataframe and work with that data uncomment the below code

# for image in df['image']:
#     image_content = image['bytes']
#     result = process_document_sample(project_id, location, processor_id, image_content , mime_type)
    
#     document_result_obj.append(result)

image_content = df.iloc[0]['image']['bytes']
result = process_document_sample(project_id, location, processor_id, image_content , mime_type)

print(time.time() - start)

8.07184386253357


In [9]:
result.text

"Invoice no: 40378170\nDate of issue:\n10/15/2012\nSeller:\nClient:\nPatel, Thompson and Montgomery\n356 Kyle Vista\nNew James, MA 46228\nJackson, Odonnell and Jackson\n267 John Track Suite 841\nJenniferville, PA 98601\nTax Id: 998-87-7723\nTax Id: 958-74-3511\nIBAN: GB77WRBQ31965128414006\nITEMS\nNo.\nDescription\nQty\nUM\nNet price\nNet worth\nVAT [%]\nGross\nworth\n1.\n1,00\neach\n7,50\n7,50\n10%\n8,25\nLeed's Wine Companion Bottle\nCorkscrew Opener Gift Box Set\nwith Foil Cutter\nSUMMARY\nVAT [%]\nNet worth\nVAT\nGross worth\n10%\n7,50\n0,75\n8,25\nTotal\n$ 7,50\n$ 0,75\n$ 8,25\n"

In [10]:
result.pages[0].tables[0].header_rows

In [11]:
result.entities[0]

In [50]:
for ent in result.entities[0].properties:
    print(f"{ent.type_} : {ent.mention_text}")

id : 40378170
date_time : 10/15/2012
person : Patel, Thompson
organization : Thompson and Montgomery
address : 356 Kyle Vista
New James, MA 46228
organization : Jackson
address : 267 John Track Suite 841
Jenniferville, PA 98601
id : 998-87-7723
id : 958-74-3511
id : GB77WRBQ31965128414006
quantity : 1,00
price : 7,50
quantity : 7,50
quantity : 10%
quantity : 8,25
organization : Wine Companion Bottle
organization : Corkscrew Opener
organization : Foil Cutter
quantity : 10%
quantity : 7,50
price : 0,75
quantity : 8,25
price : $ 7,50
price : $ 0,75
price : $ 8,25


## Extract Table using Form Parser Processor

In [12]:
from typing import Optional, Sequence
import pandas as pd


def print_table_rows(
    table_rows: Sequence[documentai.Document.Page.Table.TableRow], text: str
) -> None:
    result_list = []
    for table_row in table_rows:
        row_text = ""
        row = []
        for cell in table_row.cells:
            cell_text = layout_to_text(cell.layout, text)
            row.append(cell_text.strip())
#             row_text += f"{repr(cell_text.strip())} | "

    result_list.append(row)
    
    return result_list
        
        
def layout_to_text(layout: documentai.Document.Page.Layout, text: str) -> str:
    """
    Document AI identifies text in different parts of the document by their
    offsets in the entirety of the document"s text. This function converts
    offsets to a string.
    """
    # If a text segment spans several lines, it will
    # be stored in different text segments.
    return "".join(
        text[int(segment.start_index) : int(segment.end_index)]
        for segment in layout.text_anchor.text_segments
    )

In [13]:
text = result.text
tables = []
for page in result.pages:
    print(f"\n\n**** Page {page.page_number} ****")

    print(f"\nFound {len(page.tables)} table(s):")
    for table in page.tables:
        num_columns = len(table.header_rows[0].cells)
        num_rows = len(table.body_rows)
        print(f"Table with {num_columns} columns and {num_rows} rows:")


        header = print_table_rows(table.header_rows, text)

        rows = print_table_rows(table.body_rows, text)
        table = pd.DataFrame(rows, columns=header)
        tables.append(table)
        

        
    print(f"\nFound {len(page.form_fields)} form field(s):")
    for field in page.form_fields:
        name = layout_to_text(field.field_name, text)
        value = layout_to_text(field.field_value, text)
        print(f"    * {repr(name.strip())}: {repr(value.strip())}")



**** Page 1 ****

Found 2 table(s):
Table with 1 columns and 2 rows:
Table with 8 columns and 1 rows:

Found 11 form field(s):
    * 'Client:': 'Jackson, Odonnell and Jackson\n267 John Track Suite 841\nJenniferville, PA 98601'
    * 'Tax Id:': '998-87-7723'
    * 'Seller:': 'Patel, Thompson and Montgomery\n356 Kyle Vista\nNew James, MA 46228'
    * 'IBAN:': 'GB77WRBQ31965128414006'
    * 'Tax Id:': '958-74-3511'
    * 'Invoice no:': '40378170'
    * 'Date of issue:': '10/15/2012'
    * 'Gross\nworth': '8,25'
    * 'VAT [%]': '10%'
    * 'Net worth': '7,50'
    * 'Net price': '7,50'


#### Iterate through the resulted table list

In [15]:
print(f"Number of Tables present in the Document: {len(tables)}")

Number of Tables present in the Document: 2


In [16]:
for table in tables:
    display(table)

,VAT [%]\nNet worth\nVAT\nGross worth
0,"Total\n$ 7,50\n$ 0,75\n$ 8,25"


,No.,Description,Qty,UM,Net price,Net worth,VAT [%],Gross\nworth
0,1.,Leed's Wine Companion Bottle\nCorkscrew Opener...,"1,00",each,"7,50","7,50",10%,"8,25"


### Entities extracted using Form Parser

In [85]:
for ent in result.entities:
    if len(ent.text_anchor.text_segments) == 1:
        print(f"{ent.type_}: {ent.mention_text}")
        continue
    for prop in ent.properties:
        print(f"{prop.type_} : {prop.mention_text}")

id : 40378170
date_time : 10/15/2012
person : Patel, Thompson
organization : Thompson and Montgomery
address : 356 Kyle Vista
New James, MA 46228
organization : Jackson
address : 267 John Track Suite 841
Jenniferville, PA 98601
id : 998-87-7723
id : 958-74-3511
id : GB77WRBQ31965128414006
quantity : 1,00
price : 7,50
quantity : 7,50
quantity : 10%
quantity : 8,25
organization : Wine Companion Bottle
organization : Corkscrew Opener
organization : Foil Cutter
quantity : 10%
quantity : 7,50
price : 0,75
quantity : 8,25
price : $ 7,50
price : $ 0,75
price : $ 8,25


Entities extracted here is very general as the names of the field is not specific to invoice or any type. To improve this we can use invoice processor to extract invoice specific entities.

## Invoice

In [17]:
inv_processor_id = ""      #provide processor id here

start = time.time()

image_content = df.iloc[0]['image']['bytes']
invoice_result = process_document_sample(project_id, location, inv_processor_id, image_content , mime_type)

print(time.time() - start)

11.600863933563232


In [81]:
for ent in invoice_result.entities:
    if len(ent.text_anchor.text_segments) == 1:
        print(f"{ent.type_}: {ent.mention_text}")
        continue
    for prop in ent.properties:
        print(f"{prop.type_} : {prop.mention_text}")

vat/tax_rate : 10
vat/amount : 7,50
vat/tax_amount : 0,75
invoice_id: 40378170
invoice_date: 10/15/2012
supplier_tax_id: 958-74-3511
supplier_iban: GB77WRBQ31965128414006
total_amount: 8,25
receiver_tax_id: 998-87-7723
total_tax_amount: 0,75
receiver_name: Jackson, Odonnell and Jackson
net_amount: 7,50
supplier_address: 356 Kyle Vista
New James, MA 46228
receiver_address: 267 John Track Suite 841
Jenniferville, PA 98601
supplier_name: Patel, Thompson and Montgomery
currency: $
line_item/quantity : 1,00
line_item/unit : each
line_item/unit_price : 7,50
line_item/amount : 8,25
line_item/description : Leed's Wine Companion Bottle
Corkscrew Opener Gift Box Set
with Foil Cutter


Entities extracted from Invoice Processor are specific to invoice types. Check the above the entities. While the entities that is extracted from general form parser is very generic. It will extract the entities as such in the document.

In [31]:
json.loads(df.iloc[1]['parsed_data'])['json']

"{'header': {'invoice_no': '61356291', 'invoice_date': '09/06/2012', 'seller': 'Chapman, Kim and Green 64731 James Branch Smithmouth, NC 26872', 'client': 'Rodriguez-Stevens 2280 Angela Plain Hortonshire, MS 93248', 'seller_tax_id': '949-84-9105', 'client_tax_id': '939-98-8477', 'iban': 'GB50ACIE59715038217063'}, 'items': [{'item_desc': 'Wine Glasses Goblets Pair Clear Glass', 'item_qty': '5,00', 'item_net_price': '12,00', 'item_net_worth': '60,00', 'item_vat': '10%', 'item_gross_worth': '66,00'}, {'item_desc': 'With Hooks Stemware Storage Multiple Uses Iron Wine Rack Hanging Glass', 'item_qty': '4,00', 'item_net_price': '28,08', 'total_net_worth': '112,32', 'item_vat': '10%', 'item_gross_worth': '123,55'}, {'item_desc': 'Replacement Corkscrew Parts Spiral Worm Wine Opener Bottle Houdini', 'item_qty': '1,00', 'item_net_price': '7,50', 'total_net_worth': '7,50', 'item_vat': '10%', 'item_gross_worth': '8,25'}, {'item_desc': 'HOME ESSENTIALS GRADIENT STEMLESS WINE GLASSES SET OF 420 FL OZ

So the extracted information from the document matches the actaul data. There is difference in the name like Seller is named as Supplier when processed with document AI invoice processor which we can work on the post processing to improve the result. Otherwise the result is very good.